In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [ ]:
# Function to move along tags for better readability
def moveSibling(tag, number):
	i = 1
	while i <= number:
		tag = tag.nextSibling
		i += 1
	return tag

In [2]:
# Empty DF with the columns for the data to be downloaded.
all_seasons = pd.DataFrame(data=None, columns=["Season", "Gameday", "TeamH", "TeamA", "FTHG", "FTAG"])

for year in range(1965, 2018):
    # Skip 1992 because the season was played with 20 teams instead of 18.
    if year == 1992:
        continue
    
    url = "http://www.fussballdaten.de/bundesliga/" + str(year) + "/spielplan"
    print url

    website = requests.get(url)
    main = BeautifulSoup(website.content, 'lxml')
    
    all_games = []
    
    for gd in main.find_all('div', {'class': 'col-md-4'}):
        try:
            gameday = gd.find('div', {'class': 'box header-title'}).text
        except:
            continue
        gameday = int(float(gameday[0:2].strip()))
        match_list = gd.find('table', {'class':'table-small-spiel kv-grid-table table table-striped kv-table-wrap'})
    
        for match in match_list.find_all('a'):
            all_games.append(match.text.split(":"))
            
    # Flatten list for a given season ...
    all_games = [element for sublist in all_games for element in sublist]
    # ... and then partition into games
    all_games = [all_games[i:i+4] for i in range(0, len(all_games), 4)]

    # Store data in DF
    current_season = pd.DataFrame(data=all_games, columns=["TeamH", "TeamA", "FTHG", "FTAG"])
    # Write Season and Gameday IDs
    current_season.loc[:, "Season"] = year-1
    current_season.loc[:, "Gameday"] = (current_season.index/9)+1
    
    all_seasons = all_seasons.append(current_season)

all_seasons.to_csv("Long_History.csv", encoding='utf-8')

http://www.fussballdaten.de/bundesliga/2017/spielplan
